# CADS Catalogue API service

Scope of this document is to provide a preview of features implemented by the CADS Catalogue public API.

The CADS Catalogue is implemented by providing a [STAC Collection](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md) compliant service by representing every dataset as a collection. This is build on top of [stac-fastapi](https://github.com/stac-utils/stac-fastapi/) (reference implementation for the standard).

The root of the REST API contains information about the standard conformance itself:

In [ ]:
import requests
from tabulate import tabulate
from IPython.display import display, Markdown

API_BASE = 'http://localhost:8080/api/catalogue/v1/'

r = requests.get(API_BASE)
stac = r.json()

print(stac['stac_version'])
print(stac['conformsTo'])

To reach something more specific to the CADS catalogue, we have to introspect the `/collections` endpoint, which is a set of [standalone collections](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md#standalone-collections).

In [ ]:

r = requests.get(f"{API_BASE}/collections")
print(r.status_code)
collections = r.json()['collections']
collections

Out collections are a set of JSON structures, where every item is a dataset.
We can better display them in a table:

In [ ]:
tabulate(collections, headers='keys', tablefmt="html")

The catalogue overview is not providing all of the data we make it available through the REST API, while requesting for a specific dataset we can see more.

Let's access the _"[ERA5 hourly data on single levels from 1959 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels)"_.

In [ ]:
r = requests.get(f"{API_BASE}/collections/reanalysis-era5-single-levels")
era5 = r.json()
era5

From a dataset record we can extract a lot of informations like:

- the title…

In [ ]:
era5['title']

- description in a markdown format…

In [ ]:
display(Markdown(era5['description']))

- the preview image…

In [ ]:
display(Markdown(f'![ERA 5 Preview](http://localhost:8080{era5["assets"]["thumbnail"]["href"]})'))

Standard fields from the STAC standard can't cover all of the information we have inside datasets.

Some information are vehicolated by the use of `links` structure:

In [ ]:
era5['links']

For example, to see license(s) of the ERA5 dataset:

In [ ]:
[link['title'] for link in era5['links'] if link['rel'] == 'license']

`links` is not suitable for every type of additional data we want to provide. Adding new fields to a STAC collection is something allowed by the standard by defining a new **STAC extension**.

CADS datasets provides new data like the *publication date*:

In [ ]:
era5['tmp:publication_date']